# **Brain tumor detection**

1. Buat folder datasets
2. masuk folder yes dan no didalam folder datasets

## **Import Dependency**

In [ ]:
pip install tensorflowjs

In [37]:
import cv2
import os
from PIL import Image
from google.colab import drive
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
import tensorflowjs as tfjs

## **Mount drive**

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## **Image state**
pengumpulan image ke dalam variable

In [13]:
directory_images = "gdrive/MyDrive/BrainTumorDetection/"
without_tumor_images = os.listdir(directory_images+"datasets/"+"no/")
with_tumor_images = os.listdir(directory_images+"datasets/"+"yes/")
prediction_tumor_images = os.listdir(directory_images+"predictions/")
datasets=[]
labels=[]

In [4]:
INPUT_SIZE=64

In [5]:
for i,name in enumerate(without_tumor_images):
  if(name.split('.')[1]=="jpg"):
    image=cv2.imread(directory_images+"no/"+name)
    image=Image.fromarray(image,"RGB")
    image=image.resize((INPUT_SIZE,INPUT_SIZE))
    datasets.append(np.array(image))
    labels.append(0)

In [6]:
for i,name in enumerate(with_tumor_images):
  if(name.split('.')[1]=="jpg"):
    image=cv2.imread(directory_images+"yes/"+name)
    image=Image.fromarray(image,"RGB")
    image=image.resize((INPUT_SIZE,INPUT_SIZE))
    datasets.append(np.array(image))
    labels.append(1)

## **Build model**

In [7]:
datasets = np.array(datasets)
labels = np.array(labels)

# tf.keras.utils.normalize
x_train,x_test,y_train,y_test = train_test_split(datasets,labels,test_size=0.2,random_state=0)

In [8]:
x_train=tf.keras.utils.normalize(x_train,axis=1)
x_test=tf.keras.utils.normalize(x_test,axis=1)

y_train=to_categorical(y_train,num_classes=2)
y_test=to_categorical(y_test,num_classes=2)

In [11]:
model = tf.keras.Sequential()

model.add(Conv2D(32,(3,3),input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),kernel_initializer="he_uniform"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),kernel_initializer="he_uniform"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(x_train,y_train,batch_size=16,verbose=1,epochs=10,validation_data=(x_test,y_test),shuffle=False)

model.save("BrainTumor.h5")

Epoch 1/10
150/150 [==============================] - 3s 14ms/step - loss: 0.5263 - accuracy: 0.7437 - val_loss: 0.5136 - val_accuracy: 0.7700
Epoch 2/10
150/150 [==============================] - 2s 13ms/step - loss: 0.3882 - accuracy: 0.8333 - val_loss: 0.4184 - val_accuracy: 0.8200
Epoch 3/10
150/150 [==============================] - 2s 12ms/step - loss: 0.3055 - accuracy: 0.8712 - val_loss: 0.3302 - val_accuracy: 0.8550
Epoch 4/10
150/150 [==============================] - 2s 13ms/step - loss: 0.2353 - accuracy: 0.9054 - val_loss: 0.2528 - val_accuracy: 0.9133
Epoch 5/10
150/150 [==============================] - 2s 12ms/step - loss: 0.1678 - accuracy: 0.9333 - val_loss: 0.2039 - val_accuracy: 0.9317
Epoch 6/10
150/150 [==============================] - 2s 12ms/step - loss: 0.1100 - accuracy: 0.9600 - val_loss: 0.1845 - val_accuracy: 0.9533
Epoch 7/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0793 - accuracy: 0.9746 - val_loss: 0.1446 - val_accuracy: 0.9600

# **Test model**

In [15]:
data_model = load_model("BrainTumor.h5")

In [34]:
image_test = cv2.imread(directory_images+"predictions/"+"pred19.jpg")
img = Image.fromarray(image_test)
img = img.resize((INPUT_SIZE,INPUT_SIZE))
img = np.array(img)

input_img = np.expand_dims(img,axis=0)
result = np.argmax(data_model.predict(input_img))
# return 1 atau 0, 1=tumor,0 = tidak tumor
if(result == 1):
  print("tumor")
else:
  print("tidak tumor")

tidak tumor


# **Export model keras ke tensorflow js**

In [38]:
tfjs.converters.save_keras_model(data_model, "./")